# Monitoring setup for Bicycle Sharing Demand Prediction

This notebook shows how you can use the Evidently to:
* calculate performance and data drift for the model, performed as batch checks 
* log models quality & data drift using MLflow Tracking
* explore the result 

More examples are available in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [1]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [2]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [4]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 

In [5]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [6]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Model training 

In [7]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [8]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [9]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference[target],
    test_size=0.3
)

In [10]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [11]:
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [12]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Model validation

In [13]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [14]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [15]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=X_train.sort_index(), current_data=X_test.sort_index(),
                                  column_mapping=column_mapping)
regression_performance_report.save_html("regression_performance_report.html")

## Production model training

In [16]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [17]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [18]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [19]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=None, current_data=reference,
                                  column_mapping=column_mapping)
regression_performance_report.save_html("regression_performance_report.html")

## Some time has passed - how is the model working?

In [20]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [21]:
regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                  column_mapping=column_mapping)
regression_performance_report.save_html("regression_performance_report_week1.html")

### Week 2

In [22]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.save_html("regression_performance_report_week2.html")

### Week 3

In [23]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.save_html("regression_performance_report_week3.html")

## Why the quality has dropped?

In [24]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [25]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
    column_mapping=column_mapping_drift,
)
data_drift_report.save_html("data_drift_report.html")

## Let's customize the report!

In [26]:
from evidently.calculations.stattests import StatTest

def _anderson_stat_test(reference_data: pd.Series, current_data: pd.Series, feature_type: str, threshold: float):
    p_value = anderson_ksamp(np.array([reference_data, current_data]))[2]
    return p_value, p_value < threshold

anderson_stat_test = StatTest(
    name="anderson",
    display_name="Anderson test (p_value)",
    func=_anderson_stat_test,
    allowed_feature_types=["num"]
)

# options = DataDriftOptions(feature_stattest_func=anderson_stat_test, all_features_threshold=0.9, nbinsx=20)

In [27]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])


the_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'], 
    column_mapping=column_mapping_drift
)
the_report.save_html("customized_report.html")

## Jupyter vs Automated Run?

To run this part of the tutorial you need to install MLflow (if not installed yet)

You can install MLflow with the following command: `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`

More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

In [28]:
try:
    import mlflow
except:
    !pip install mlflow

In [29]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [30]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [31]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])

the_report.run(
    reference_data=reference, 
    current_data=current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
    column_mapping=column_mapping_drift
)

In [32]:
logged_json = json.loads(the_report.json())

In [33]:
logged_json

{'version': '0.4.7',
 'metrics': [{'metric': 'RegressionQualityMetric',
   'result': {'columns': {'utility_columns': {'date': None,
      'id': None,
      'target': 'cnt',
      'prediction': 'prediction'},
     'num_feature_names': ['atemp',
      'hr',
      'hum',
      'mnth',
      'temp',
      'weekday',
      'windspeed'],
     'cat_feature_names': [],
     'text_feature_names': [],
     'datetime_feature_names': ['dteday'],
     'target_names': None},
    'current': {'r2_score': 0.8342907435814624,
     'rmse': 21.03964953999846,
     'mean_error': -6.26536170212766,
     'mean_abs_error': 13.52936170212766,
     'mean_abs_perc_error': 41.57714593080225,
     'abs_error_max': 94.34,
     'underperformance': {'majority': {'mean_error': -4.857251184834124,
       'std_error': 12.75642799788315},
      'underestimation': {'mean_error': -65.01666666666665,
       'std_error': 19.82673462929804},
      'overestimation': {'mean_error': 27.726666666666663,
       'std_error': 7.3900

In [34]:
[x['metric'] for x in logged_json['metrics']]

['RegressionQualityMetric',
 'RegressionErrorPlot',
 'RegressionErrorDistribution',
 'DatasetDriftMetric',
 'DataDriftTable']

In [35]:
logged_json['metrics'][0]['result']['current']['mean_error']

-6.26536170212766

In [36]:
logged_json['metrics'][3]['result']['drift_share']

0.5

In [37]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Model Quality Evaluation')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        the_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution(),
            DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
        ])
        the_report.run(
            reference_data=reference, 
            current_data=current.loc[date[0]:date[1]],
            column_mapping=column_mapping_drift)
        logged_json = json.loads(the_report.json())
        
        me = logged_json['metrics'][0]['result']['current']['mean_error']
        mae = logged_json['metrics'][0]['result']['current']["mean_abs_error"]
        drift_share = logged_json['metrics'][3]['result']['drift_share']
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        mlflow.log_metric('drift_share', round(drift_share, 3))

        print(run.info)

2023/11/20 21:24:46 INFO mlflow.tracking.fluent: Experiment with name 'Model Quality Evaluation' does not exist. Creating a new experiment.


<RunInfo: artifact_uri='file:///Users/migumax/Documents/My_projects/Udemy/MLOps/updates/202311/practice_python/notebooks/mlruns/866644469432695052/d71385f94a3a419db7c21f9bcc5cd546/artifacts', end_time=None, experiment_id='866644469432695052', lifecycle_stage='active', run_id='d71385f94a3a419db7c21f9bcc5cd546', run_name='marvelous-crab-982', run_uuid='d71385f94a3a419db7c21f9bcc5cd546', start_time=1700511886432, status='RUNNING', user_id='migumax'>
<RunInfo: artifact_uri='file:///Users/migumax/Documents/My_projects/Udemy/MLOps/updates/202311/practice_python/notebooks/mlruns/866644469432695052/4801070947e542db93557e6895be8726/artifacts', end_time=None, experiment_id='866644469432695052', lifecycle_stage='active', run_id='4801070947e542db93557e6895be8726', run_name='dashing-smelt-445', run_uuid='4801070947e542db93557e6895be8726', start_time=1700511886603, status='RUNNING', user_id='migumax'>
<RunInfo: artifact_uri='file:///Users/migumax/Documents/My_projects/Udemy/MLOps/updates/202311/prac

In [ ]:
#run MLflow UI (NOT recommended! It will be more convinient to run it directly from the TERMINAL)
# !mlflow ui